This notebook explores a machine learning algorithm to predict the stock prices of SPY, the S&P 500 ETF, and is intended to utilize functions that can be easily translated to a python executable.

In [1]:
# installing dependencies
!pip install numpy
!pip install pandas
!pip install torch
!pip install matplotlib
!pip install alpha_vantage
!pip install scikit-learn
!pip install pandas_market_calendars
!pip install lxml

In [2]:
# import libararies
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from alpha_vantage.timeseries import TimeSeries 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import pandas_market_calendars as mcal
import time

In [3]:
# config file (placing here for now, some fields will change on later impplementations)

config = {
    "alpha_vantage": {
        "key": "2JMCN347HZ3BU9RC", 
        "symbol": "SPY",
        "outputsize": "full",
        "key_adjusted_close": "5. adjusted close",
    },
    "data": {
        "window_size": 30,
        "train_split_size": 1,
    }, 
    "plots": {
        "xticks_interval": 90, # show a date every 90 days
        "color_actual": "#001f3f",
        "color_train": "#3D9970",
        "color_val": "#0074D9",
        "color_pred_train": "#3D9970",
        "color_pred_val": "#0074D9",
        "color_pred_test": "#FF4136",
    },
    "model": {
        "input_size": 1, # since for now we are only using close price
        "num_lstm_layers": 2,
        "lstm_size": 32,
        "dropout": 0.2,
    },
    "training": {
        "device": "cpu",
        "batch_size": 64,
        "num_epoch": 100,
        "epoch_stop": 10,
        "learning_rate": 0.01,
        "scheduler_step_size": 40,
    }
}

In [4]:
# sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
# tickers = np.array(sp500[0]['Symbol'])
# if ('BF.B' in tickers)
#     tickers[tickers.index('BF.B')] = 'BF-B'

start = time.time()

tickers = ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP',
       'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE',
       'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN',
       'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK',
       'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA',
       'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO',
       'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX',
       'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK',
       'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO',
       'BF-B', 'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH',
       'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE',
       'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG',
       'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX',
       'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP',
       'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST',
       'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA',
       'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DISH',
       'DIS', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DUK', 'DD',
       'DXC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'ELV',
       'LLY', 'EMR', 'ENPH', 'ETR', 'EOG', 'EPAM', 'EQT', 'EFX', 'EQIX',
       'EQR', 'ESS', 'EL', 'ETSY', 'RE', 'EVRG', 'ES', 'EXC', 'EXPE',
       'EXPD', 'EXR', 'XOM', 'FFIV', 'FDS', 'FICO', 'FAST', 'FRT', 'FDX',
       'FITB', 'FRC', 'FSLR', 'FE', 'FIS', 'FISV', 'FLT', 'FMC', 'F',
       'FTNT', 'FTV', 'FOXA', 'FOX', 'BEN', 'FCX', 'GRMN', 'IT', 'GEHC',
       'GEN', 'GNRC', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD', 'GL', 'GPN',
       'GS', 'HAL', 'HIG', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES',
       'HPE', 'HLT', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HWM', 'HPQ',
       'HUM', 'HBAN', 'HII', 'IBM', 'IEX', 'IDXX', 'ITW', 'ILMN', 'INCY',
       'IR', 'PODD', 'INTC', 'ICE', 'IFF', 'IP', 'IPG', 'INTU', 'ISRG',
       'IVZ', 'INVH', 'IQV', 'IRM', 'JBHT', 'JKHY', 'J', 'JNJ', 'JCI',
       'JPM', 'JNPR', 'K', 'KDP', 'KEY', 'KEYS', 'KMB', 'KIM', 'KMI',
       'KLAC', 'KHC', 'KR', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LDOS',
       'LEN', 'LNC', 'LIN', 'LYV', 'LKQ', 'LMT', 'L', 'LOW', 'LYB', 'MTB',
       'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MTCH',
       'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'META', 'MET', 'MTD', 'MGM',
       'MCHP', 'MU', 'MSFT', 'MAA', 'MRNA', 'MHK', 'MOH', 'TAP', 'MDLZ',
       'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'NDAQ', 'NTAP',
       'NFLX', 'NWL', 'NEM', 'NWSA', 'NWS', 'NEE', 'NKE', 'NI', 'NDSN',
       'NSC', 'NTRS', 'NOC', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'NXPI',
       'ORLY', 'OXY', 'ODFL', 'OMC', 'ON', 'OKE', 'ORCL', 'OGN', 'OTIS',
       'PCAR', 'PKG', 'PARA', 'PH', 'PAYX', 'PAYC', 'PYPL', 'PNR', 'PEP',
       'PKI', 'PFE', 'PCG', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'POOL',
       'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PTC',
       'PSA', 'PHM', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTX',
       'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RHI', 'ROK', 'ROL', 'ROP',
       'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SEE', 'SRE',
       'NOW', 'SHW', 'SPG', 'SWKS', 'SJM', 'SNA', 'SEDG', 'SO', 'LUV',
       'SWK', 'SBUX', 'STT', 'STLD', 'STE', 'SYK', 'SYF', 'SNPS', 'SYY',
       'TMUS', 'TROW', 'TTWO', 'TPR', 'TRGP', 'TGT', 'TEL', 'TDY', 'TFX',
       'TER', 'TSLA', 'TXN', 'TXT', 'TMO', 'TJX', 'TSCO', 'TT', 'TDG',
       'TRV', 'TRMB', 'TFC', 'TYL', 'TSN', 'USB', 'UDR', 'ULTA', 'UNP',
       'UAL', 'UPS', 'URI', 'UNH', 'UHS', 'VLO', 'VTR', 'VRSN', 'VRSK',
       'VZ', 'VRTX', 'VFC', 'VTRS', 'VICI', 'V', 'VMC', 'WAB', 'WBA',
       'WMT', 'WBD', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WST', 'WDC',
       'WRK', 'WY', 'WHR', 'WMB', 'WTW', 'GWW', 'WYNN', 'XEL', 'XYL',
       'YUM', 'ZBRA', 'ZBH', 'ZION', 'ZTS']

In [5]:
# to account for API limitations (on the weekend)

# index = tickers.index('REG')
# tickers = tickers[(index+1):]
len(tickers)

503

In [6]:
today = datetime.now()
next_weeks = mcal.date_range(mcal.get_calendar('NYSE').schedule(start_date=today, end_date=(today+relativedelta(months=1))), frequency='1D')
next_weeks = [date.strftime('%Y-%m-%d') for date in next_weeks]
next_weeks = next_weeks[1:]
next_day = next_weeks[0]
next_day

'2023-04-03'

# Classes and Functions

In [7]:
# get data from the configuration file
def get_data(config, ticker):
    ts = TimeSeries(key=config["alpha_vantage"]["key"]) 
    data, meta_data = ts.get_daily_adjusted(ticker, outputsize=config["alpha_vantage"]["outputsize"])

    date_data = [date for date in data.keys()]
    date_data.reverse()

    close_price_data = [float(data[date][config["alpha_vantage"]["key_adjusted_close"]]) for date in data.keys()]
    close_price_data.reverse()
    close_price_data = np.array(close_price_data)

    num_data_points = len(date_data)
    display_date_range = "from " + date_data[0] + " to " + date_data[num_data_points-1]
    print("Number data points", num_data_points, display_date_range)

    return date_data, close_price_data, num_data_points, display_date_range

class Normalization():
    def __init__(self):
        self.mu = None
        self.sd = None

    def fit_transform(self, x):
        self.mu = np.mean(x, axis=(0), keepdims=True)
        self.sd = np.std(x, axis=(0), keepdims=True)
        normalized_x = (x - self.mu)/self.sd
        return normalized_x

    def inverse_transform(self, x):
        return (x*self.sd) + self.mu
    
def prepare_data_x(x, window_size):
    # perform windowing
    n_row = x.shape[0] - window_size + 1
    output = np.lib.stride_tricks.as_strided(x, shape=(n_row, window_size), strides=(x.strides[0], x.strides[0]))
    return output[:-1], output[-1]


def prepare_data_y(x, window_size):
    # use the next day as label
    output = x[window_size:]
    return output

# Class to prepare data for training and LSTM model
class TimeSeriesDataset(Dataset):
    def __init__(self, x, y):
        x = np.expand_dims(x, 2) # right now we have only 1 feature, so we need to convert `x` into [batch, sequence, features]
        self.x = x.astype(np.float32)
        self.y = y.astype(np.float32)
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

# neural network model definition
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=32, num_layers=2, output_size=1, dropout=0.2):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.linear_1 = nn.Linear(input_size, hidden_layer_size)
        self.relu = nn.ReLU()
        self.lstm = nn.LSTM(hidden_layer_size, hidden_size=self.hidden_layer_size, num_layers=num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(num_layers*hidden_layer_size, output_size)
        
        self.init_weights()

    def init_weights(self):
        for name, param in self.lstm.named_parameters():
            if 'bias' in name:
                 nn.init.constant_(param, 0.0)
            elif 'weight_ih' in name:
                 nn.init.kaiming_normal_(param)
            elif 'weight_hh' in name:
                 nn.init.orthogonal_(param)

    def forward(self, x):
        batchsize = x.shape[0]

        # layer 1
        x = self.linear_1(x)
        x = self.relu(x)
        
        # LSTM layer
        lstm_out, (h_n, c_n) = self.lstm(x)

        # reshape output from hidden cell into [batch, features] for `linear_2`
        x = h_n.permute(1, 0, 2).reshape(batchsize, -1) 
        
        # layer 2
        x = self.dropout(x)
        predictions = self.linear_2(x)
        return predictions[:,-1]
    
# function for training LSTM model
def run_epoch(dataloader, is_training=False):
    epoch_loss = 0

    if is_training:
        model.train()
    else:
        model.eval()

    for idx, (x, y) in enumerate(dataloader):
        if is_training:
            optimizer.zero_grad()

        batchsize = x.shape[0]

        x = x.to(config["training"]["device"])
        y = y.to(config["training"]["device"])

        out = model(x)
        loss = criterion(out.contiguous(), y.contiguous())

        if is_training:
            loss.backward()
            optimizer.step()

        epoch_loss += (loss.detach().item() / batchsize)

    lr = scheduler.get_last_lr()[0]

    return epoch_loss, lr


## Predict the Next Day

In [8]:
# API call limit per minute
limit = 5

check = time.time()

for i,ticker in enumerate(tickers):
    
    # account for API calls per minute
    if not i + 1 % limit:
        sleep = check + 60 - time.time()
        if sleep > 0:
            time.sleep(sleep)
        check = time.time()      
    
    date_data, close_price_data, num_data_points, display_date_range = get_data(config, ticker)
    
    scaler = Normalization()
    normalized_close_price_data = scaler.fit_transform(close_price_data)
    
    data_x, data_x_unseen = prepare_data_x(normalized_close_price_data, window_size=config["data"]["window_size"])
    data_y = prepare_data_y(normalized_close_price_data, window_size=config["data"]["window_size"])
    
    split_index = int(data_y.shape[0]*config["data"]["train_split_size"])
    data_x_train = data_x[:split_index]
    data_x_val = data_x[split_index:]
    data_y_train = data_y[:split_index]
    data_y_val = data_y[split_index:]

    train_dataset = TimeSeriesDataset(data_x_train, data_y_train)
    train_dataloader = DataLoader(train_dataset, batch_size=config["training"]["batch_size"], shuffle=True)

    model = LSTMModel(input_size=config["model"]["input_size"], hidden_layer_size=config["model"]["lstm_size"], num_layers=config["model"]["num_lstm_layers"], output_size=1, dropout=config["model"]["dropout"])
    model = model.to(config["training"]["device"])

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=config["training"]["learning_rate"], betas=(0.9, 0.98), eps=1e-9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=config["training"]["scheduler_step_size"], gamma=0.1)

    best_loss = np.inf
    epochs_no_improve = 0
    n_epochs_stop = config["training"]["epoch_stop"]
    
    print('{} training'.format(ticker))
    for epoch in range(config["training"]["num_epoch"]):
        loss_train, lr_train = run_epoch(train_dataloader, is_training=True)
        scheduler.step()

        print('Epoch[{}/{}] | loss train:{:.6f}| lr:{:.6f}'
                  .format(epoch+1, config["training"]["num_epoch"], loss_train, lr_train))
        if loss_train < best_loss:
            best_loss = loss_train
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if epochs_no_improve == n_epochs_stop:
            print("Early stopping.")
            break
            
    model.eval()
    
    torch.tensor(data_x_unseen)
    x = torch.tensor(data_x_unseen).float().to(config["training"]["device"]).unsqueeze(0).unsqueeze(2) # this is the data type and shape required, [batch, sequence, feature]
    prediction = model(x)
    prediction = prediction.cpu().detach().numpy()
    prediction[0] = scaler.inverse_transform(prediction[0])
    
    df = pd.DataFrame([[next_day,prediction[0]]], columns = ['date','close price']) 
    path = "csv/TimeSeries/" + ticker + "_predict.csv"
    df.to_csv(path)


Number data points 5892 from 1999-11-01 to 2023-03-31
MMM training
Epoch[1/100] | loss train:0.066839| lr:0.010000
Epoch[2/100] | loss train:0.015922| lr:0.010000
Epoch[3/100] | loss train:0.011171| lr:0.010000
Epoch[4/100] | loss train:0.010863| lr:0.010000
Epoch[5/100] | loss train:0.010050| lr:0.010000
Epoch[6/100] | loss train:0.010715| lr:0.010000
Epoch[7/100] | loss train:0.009738| lr:0.010000
Epoch[8/100] | loss train:0.009718| lr:0.010000
Epoch[9/100] | loss train:0.010068| lr:0.010000
Epoch[10/100] | loss train:0.009039| lr:0.010000
Epoch[11/100] | loss train:0.008981| lr:0.010000
Epoch[12/100] | loss train:0.009948| lr:0.010000
Epoch[13/100] | loss train:0.009361| lr:0.010000
Epoch[14/100] | loss train:0.009384| lr:0.010000
Epoch[15/100] | loss train:0.009316| lr:0.010000
Epoch[16/100] | loss train:0.008631| lr:0.010000
Epoch[17/100] | loss train:0.008655| lr:0.010000
Epoch[18/100] | loss train:0.009768| lr:0.010000
Epoch[19/100] | loss train:0.008094| lr:0.010000
Epoch[20/10

Epoch[19/100] | loss train:0.010119| lr:0.010000
Epoch[20/100] | loss train:0.008708| lr:0.010000
Epoch[21/100] | loss train:0.008511| lr:0.010000
Epoch[22/100] | loss train:0.008792| lr:0.010000
Epoch[23/100] | loss train:0.008749| lr:0.010000
Epoch[24/100] | loss train:0.009086| lr:0.010000
Epoch[25/100] | loss train:0.008485| lr:0.010000
Early stopping.
Number data points 5892 from 1999-11-01 to 2023-03-31
ATVI training
Epoch[1/100] | loss train:0.049947| lr:0.010000
Epoch[2/100] | loss train:0.015757| lr:0.010000
Epoch[3/100] | loss train:0.012745| lr:0.010000
Epoch[4/100] | loss train:0.012041| lr:0.010000
Epoch[5/100] | loss train:0.011534| lr:0.010000
Epoch[6/100] | loss train:0.009840| lr:0.010000
Epoch[7/100] | loss train:0.011920| lr:0.010000
Epoch[8/100] | loss train:0.009424| lr:0.010000
Epoch[9/100] | loss train:0.010734| lr:0.010000
Epoch[10/100] | loss train:0.009813| lr:0.010000
Epoch[11/100] | loss train:0.009702| lr:0.010000
Epoch[12/100] | loss train:0.009787| lr:0.0

Epoch[5/100] | loss train:0.010733| lr:0.010000
Epoch[6/100] | loss train:0.011522| lr:0.010000
Epoch[7/100] | loss train:0.008962| lr:0.010000
Epoch[8/100] | loss train:0.010467| lr:0.010000
Epoch[9/100] | loss train:0.010704| lr:0.010000
Epoch[10/100] | loss train:0.011225| lr:0.010000
Epoch[11/100] | loss train:0.009392| lr:0.010000
Epoch[12/100] | loss train:0.010559| lr:0.010000
Epoch[13/100] | loss train:0.010198| lr:0.010000
Epoch[14/100] | loss train:0.011358| lr:0.010000
Epoch[15/100] | loss train:0.009343| lr:0.010000
Epoch[16/100] | loss train:0.008920| lr:0.010000
Epoch[17/100] | loss train:0.010111| lr:0.010000
Epoch[18/100] | loss train:0.009273| lr:0.010000
Epoch[19/100] | loss train:0.009118| lr:0.010000
Epoch[20/100] | loss train:0.010695| lr:0.010000
Epoch[21/100] | loss train:0.009544| lr:0.010000
Epoch[22/100] | loss train:0.009732| lr:0.010000
Epoch[23/100] | loss train:0.009204| lr:0.010000
Epoch[24/100] | loss train:0.009481| lr:0.010000
Epoch[25/100] | loss trai

Epoch[44/100] | loss train:0.008126| lr:0.001000
Epoch[45/100] | loss train:0.007716| lr:0.001000
Epoch[46/100] | loss train:0.008480| lr:0.001000
Epoch[47/100] | loss train:0.008044| lr:0.001000
Epoch[48/100] | loss train:0.008520| lr:0.001000
Epoch[49/100] | loss train:0.008429| lr:0.001000
Epoch[50/100] | loss train:0.008177| lr:0.001000
Epoch[51/100] | loss train:0.007970| lr:0.001000
Epoch[52/100] | loss train:0.008431| lr:0.001000
Epoch[53/100] | loss train:0.007901| lr:0.001000
Epoch[54/100] | loss train:0.008531| lr:0.001000
Epoch[55/100] | loss train:0.007860| lr:0.001000
Early stopping.
Number data points 5892 from 1999-11-01 to 2023-03-31
APD training
Epoch[1/100] | loss train:0.069953| lr:0.010000
Epoch[2/100] | loss train:0.015306| lr:0.010000
Epoch[3/100] | loss train:0.013972| lr:0.010000
Epoch[4/100] | loss train:0.016143| lr:0.010000
Epoch[5/100] | loss train:0.012007| lr:0.010000
Epoch[6/100] | loss train:0.011785| lr:0.010000
Epoch[7/100] | loss train:0.011562| lr:0.

Epoch[9/100] | loss train:0.013472| lr:0.010000
Epoch[10/100] | loss train:0.013160| lr:0.010000
Epoch[11/100] | loss train:0.013473| lr:0.010000
Epoch[12/100] | loss train:0.012992| lr:0.010000
Epoch[13/100] | loss train:0.012012| lr:0.010000
Epoch[14/100] | loss train:0.013528| lr:0.010000
Epoch[15/100] | loss train:0.013733| lr:0.010000
Epoch[16/100] | loss train:0.015446| lr:0.010000
Epoch[17/100] | loss train:0.013408| lr:0.010000
Epoch[18/100] | loss train:0.012849| lr:0.010000
Epoch[19/100] | loss train:0.011602| lr:0.010000
Epoch[20/100] | loss train:0.014357| lr:0.010000
Epoch[21/100] | loss train:0.011607| lr:0.010000
Epoch[22/100] | loss train:0.011766| lr:0.010000
Epoch[23/100] | loss train:0.012279| lr:0.010000
Epoch[24/100] | loss train:0.011966| lr:0.010000
Epoch[25/100] | loss train:0.011561| lr:0.010000
Epoch[26/100] | loss train:0.013027| lr:0.010000
Epoch[27/100] | loss train:0.012594| lr:0.010000
Epoch[28/100] | loss train:0.011664| lr:0.010000
Epoch[29/100] | loss 

...

Epoch[47/100] | loss train:0.006814| lr:0.001000
Epoch[48/100] | loss train:0.006595| lr:0.001000
Epoch[49/100] | loss train:0.006752| lr:0.001000
Epoch[50/100] | loss train:0.006382| lr:0.001000
Epoch[51/100] | loss train:0.006685| lr:0.001000
Epoch[52/100] | loss train:0.006789| lr:0.001000
Epoch[53/100] | loss train:0.006645| lr:0.001000
Epoch[54/100] | loss train:0.006864| lr:0.001000
Epoch[55/100] | loss train:0.006830| lr:0.001000
Epoch[56/100] | loss train:0.006758| lr:0.001000
Epoch[57/100] | loss train:0.006748| lr:0.001000
Epoch[58/100] | loss train:0.006725| lr:0.001000
Epoch[59/100] | loss train:0.006692| lr:0.001000
Epoch[60/100] | loss train:0.006619| lr:0.001000
Early stopping.
Number data points 5892 from 1999-11-01 to 2023-03-31
WFC training
Epoch[1/100] | loss train:0.071379| lr:0.010000
Epoch[2/100] | loss train:0.016320| lr:0.010000
Epoch[3/100] | loss train:0.012754| lr:0.010000
Epoch[4/100] | loss train:0.012619| lr:0.010000
Epoch[5/100] | loss train:0.011815| lr:

Epoch[60/100] | loss train:0.007059| lr:0.001000
Epoch[61/100] | loss train:0.007822| lr:0.001000
Epoch[62/100] | loss train:0.007692| lr:0.001000
Epoch[63/100] | loss train:0.007605| lr:0.001000
Epoch[64/100] | loss train:0.007583| lr:0.001000
Epoch[65/100] | loss train:0.006963| lr:0.001000
Epoch[66/100] | loss train:0.007232| lr:0.001000
Epoch[67/100] | loss train:0.006739| lr:0.001000
Early stopping.
Number data points 5892 from 1999-11-01 to 2023-03-31
WDC training
Epoch[1/100] | loss train:0.066017| lr:0.010000
Epoch[2/100] | loss train:0.017967| lr:0.010000
Epoch[3/100] | loss train:0.015494| lr:0.010000
Epoch[4/100] | loss train:0.013162| lr:0.010000
Epoch[5/100] | loss train:0.012560| lr:0.010000
Epoch[6/100] | loss train:0.012015| lr:0.010000
Epoch[7/100] | loss train:0.011563| lr:0.010000
Epoch[8/100] | loss train:0.011678| lr:0.010000
Epoch[9/100] | loss train:0.010761| lr:0.010000
Epoch[10/100] | loss train:0.012866| lr:0.010000
Epoch[11/100] | loss train:0.012124| lr:0.01

Epoch[2/100] | loss train:0.017145| lr:0.010000
Epoch[3/100] | loss train:0.015196| lr:0.010000
Epoch[4/100] | loss train:0.015076| lr:0.010000
Epoch[5/100] | loss train:0.014378| lr:0.010000
Epoch[6/100] | loss train:0.012847| lr:0.010000
Epoch[7/100] | loss train:0.013697| lr:0.010000
Epoch[8/100] | loss train:0.012728| lr:0.010000
Epoch[9/100] | loss train:0.011636| lr:0.010000
Epoch[10/100] | loss train:0.012198| lr:0.010000
Epoch[11/100] | loss train:0.012856| lr:0.010000
Epoch[12/100] | loss train:0.012479| lr:0.010000
Epoch[13/100] | loss train:0.012614| lr:0.010000
Epoch[14/100] | loss train:0.012483| lr:0.010000
Epoch[15/100] | loss train:0.010921| lr:0.010000
Epoch[16/100] | loss train:0.011086| lr:0.010000
Epoch[17/100] | loss train:0.011947| lr:0.010000
Epoch[18/100] | loss train:0.012111| lr:0.010000
Epoch[19/100] | loss train:0.012084| lr:0.010000
Epoch[20/100] | loss train:0.011596| lr:0.010000
Epoch[21/100] | loss train:0.011910| lr:0.010000
Epoch[22/100] | loss train:0

Epoch[9/100] | loss train:0.009489| lr:0.010000
Epoch[10/100] | loss train:0.008537| lr:0.010000
Epoch[11/100] | loss train:0.009903| lr:0.010000
Epoch[12/100] | loss train:0.009427| lr:0.010000
Epoch[13/100] | loss train:0.009026| lr:0.010000
Epoch[14/100] | loss train:0.010059| lr:0.010000
Epoch[15/100] | loss train:0.009262| lr:0.010000
Epoch[16/100] | loss train:0.009154| lr:0.010000
Epoch[17/100] | loss train:0.008695| lr:0.010000
Epoch[18/100] | loss train:0.008845| lr:0.010000
Epoch[19/100] | loss train:0.009527| lr:0.010000
Epoch[20/100] | loss train:0.008186| lr:0.010000
Epoch[21/100] | loss train:0.009947| lr:0.010000
Epoch[22/100] | loss train:0.008422| lr:0.010000
Epoch[23/100] | loss train:0.009627| lr:0.010000
Epoch[24/100] | loss train:0.008127| lr:0.010000
Epoch[25/100] | loss train:0.009069| lr:0.010000
Epoch[26/100] | loss train:0.008524| lr:0.010000
Epoch[27/100] | loss train:0.007947| lr:0.010000
Epoch[28/100] | loss train:0.008978| lr:0.010000
Epoch[29/100] | loss 

Number data points 5892 from 1999-11-01 to 2023-03-31
ZION training
Epoch[1/100] | loss train:0.070256| lr:0.010000
Epoch[2/100] | loss train:0.016726| lr:0.010000
Epoch[3/100] | loss train:0.015300| lr:0.010000
Epoch[4/100] | loss train:0.014830| lr:0.010000
Epoch[5/100] | loss train:0.015252| lr:0.010000
Epoch[6/100] | loss train:0.013425| lr:0.010000
Epoch[7/100] | loss train:0.014977| lr:0.010000
Epoch[8/100] | loss train:0.014787| lr:0.010000
Epoch[9/100] | loss train:0.013546| lr:0.010000
Epoch[10/100] | loss train:0.014386| lr:0.010000
Epoch[11/100] | loss train:0.014236| lr:0.010000
Epoch[12/100] | loss train:0.014198| lr:0.010000
Epoch[13/100] | loss train:0.013357| lr:0.010000
Epoch[14/100] | loss train:0.012813| lr:0.010000
Epoch[15/100] | loss train:0.014623| lr:0.010000
Epoch[16/100] | loss train:0.014279| lr:0.010000
Epoch[17/100] | loss train:0.014267| lr:0.010000
Epoch[18/100] | loss train:0.013484| lr:0.010000
Epoch[19/100] | loss train:0.013663| lr:0.010000
Epoch[20/1

In [9]:
end = time.time()
print("Program finished in {} seconds".format(end-start))

Program finished in 12856.391211271286 seconds


In [10]:
predictions = pd.DataFrame(columns = ['date','ticker','close price'])

for ticker in tickers:
    path = "csv/TimeSeries/" + ticker + "_predict.csv"
    df1 = pd.read_csv(path)
    data = {'date': [df1['date'][0]],
            'ticker': [ticker],
            'close price': [df1['close price'][0]]}
    df2 = pd.DataFrame(data)
    predictions = pd.concat([predictions,df2], ignore_index=True)
    
predictions.to_csv("csv/predictions.csv")